In [10]:
# Library import
import xml.etree.ElementTree as ET
import spacy
import string
import numpy as np
import json
import random
import pickle
# for auto-correct words in review text
from autocorrect import Speller
spell = Speller()

In [11]:
# loading vocabulary of training data
with open('token_to_index_laptops_sentence_level.json', 'rb') as f:
    token_to_index = pickle.load(f)

max_sent_length = 75

In [12]:
xml_filepath = './dataset/EN_LAPT_SB1_TEST_.xml.gold'
# create element tree object
tree = ET.parse(xml_filepath)

# get root element
root = tree.getroot()

In [13]:
data = []
for it1 in root.findall('Review'):
    for it2 in it1.findall('sentences'):
        for it3 in it2.findall('sentence'):
            for it4 in it3.findall('text'):
                review_text = spell(it4.text.lower())

            polarity_sum = 0
            for it4 in it3.findall('Opinions'):
                for it5 in it4.findall('Opinion'):
                    if it5.attrib['polarity'] == 'negative':
                        polarity_sum += -1
                    elif it5.attrib['polarity'] == 'neutral':
                        polarity_sum += 0
                    else:
                        polarity_sum += 1
                    
            # 0 negtaive, 1 neutral, 2 positive
            if polarity_sum == 0:
                polarity = 1  #neutral
            elif polarity_sum > 0:
                polarity = 2  #positive
            else:
                polarity = 0  #negative

            data.append([review_text,polarity])

In [14]:
print(len(data))

808


In [15]:
nlp = spacy.load("en_core_web_lg")

In [16]:
# removed punctuation
for i in range(len(data)):
    sent = data[i][0]
    token_lst = []
    for it in nlp(sent):
        token = it.text
        str = ''
        for ch in token:
            if ch == "'" or ch not in string.punctuation:
                if ch == "'":
                    str += ' '
                str += ch
            else:
                str += ' '
                
        lst = str.split()
        for token in lst:
            if token in token_to_index:
                token_lst.append(token)
            else:
                token_lst.append('oov')

    data[i][0] = token_lst

In [17]:
# sentence level padding and converting to indices
for i in range(len(data)):
    sent_ind = []
    for j in range(max_sent_length):
        if j < len(data[i][0]):
            sent_ind.append(token_to_index[data[i][0][j]])
        else:
            sent_ind.append(token_to_index['pad'])
    data[i][0] = sent_ind

In [18]:
dump_this_dict = {}
for i in range(len(data)):
    dump_this_dict[i] = data[i]

In [19]:
with open('test_laptops_sentence_level.json', 'wb') as f:
    pickle.dump(dump_this_dict, f)

In [20]:
with open('test_laptops_sentence_level.json', 'rb') as f:
    data = pickle.load(f)

In [21]:
total = 0
pos = 0
neg = 0
neu = 0

for i in range(len(data)):
    if data[i][1] == 0:
        neg += 1
    elif data[i][1] == 1:
        neu += 1
    else:
        pos += 1

    total += 1

print(total, pos, neg, neu)

808 334 198 276
